# Initialize PyImageJ

In [1]:
# Import module
import jpype
# Enable Java imports
import jpype.imports
# Pull in types
from jpype.types import *

import scyjava as sj # scyjava : Supercharged Java access from Python, see https://github.com/scijava/scyjava
import imagej

# Configurations
# NOTE: The ImageJ2 gateway is initialized through a Java Virtual Machine (JVM). If you want to configure the JVM, it must be done before initializing an ImageJ2 gateway.
# sj.config.add_option('-Xmx10g') # adjust memory available to Java
sj.config.endpoints.append('ome:formats-gpl:6.11.1')

In [2]:
Fiji_Local = r"C:\Users\confocal_microscope\Desktop\Tools\Fiji.app"

# ij = imagej.init(Fiji_Local) # Same as "ij = imagej.init(Fiji_Local, mode='headless')", PyImageJ’s default mode is headless
# ij = imagej.init(Fiji_Local, mode='gui') # GUI mode (會卡在這一行 -> blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#gui-mode
ij = imagej.init(Fiji_Local, mode='interactive') # Interactive mode (可以繼續向下執行 -> non-blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#interactive-mode
ij.ui().showUI() # display the Fiji GUI

print(ij.getApp().getInfo(True)) # ImageJ2 2.9.0/1.54b

ImageJ2 2.9.0/1.54b; Java 1.8.0_362 [amd64]; 154MB of 14542MB


# Import Customized modules and Create ImageJ Object

In [3]:
import os
import sys
import re
from typing import TextIO
from datetime import datetime

from glob import glob
import logging
from skimage.io import imread, imshow, imsave


sys.path.append(r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\modules") # add path to scan customized module

from logger import init_logger
from fileop import create_new_dir
from zebrafish_lifcheck import check_image_name
from composite_ij import dump_info, median_R1_and_mean3D_R2, save_tif_with_SN, group_show, group_hide, crop_threshold_rect, create_rect

In [4]:
# Bio-Format Reader
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

# [IMPORTANT] some ImageJ plugins need to be new before use
from ij.plugin.frame import RoiManager
from ij.plugin import ImageCalculator
from ij.plugin import ChannelSplitter
from ij.plugin import ZProjector
from ij.plugin import RGBStackMerge
from ij.plugin import RGBStackConverter

rm = RoiManager()
imageCalculator = ImageCalculator()
channelSplitter = ChannelSplitter()
zProjector = ZProjector()
rgbStackMerge = RGBStackMerge()
rgbStackConverter = RGBStackConverter()

In [5]:
log = init_logger(r"{Test}_RGB_preprocess")

Test : RGB image name 

In [6]:
# test_string = r"Series001_fish_1_palmskin_8dpf_A"
test_string = r"Series001_fish_165_A_RG"


# Check format of 'image_name'
detect_old_new_list = re.findall("[RGB]", test_string)
# print(detect_old_new_list)
image_name_list = re.split(" |_|-", test_string)
if len(detect_old_new_list) > 0: failed_cnt, check_dict = check_image_name(image_name_list, "new_rgb", debug_mode=True)
else: failed_cnt, check_dict = check_image_name(image_name_list, "old_rgb", debug_mode=True)


# Consociate image_name
if (check_dict["    format_and_type     "] == "old_rgb") and (failed_cnt == 0): # name check passed, generate new name
    image_name_list.pop(3) # palmskin (old name only)
    image_name_list.pop(3) # [num]dpf (old name only)
    image_name_list.append("RGB")
image_name = "_".join(image_name_list)


# log.info() --> ERRORs when checking 'image_name'
if failed_cnt == -1 : log.info(check_dict["  len(image_name_list)  "])
if failed_cnt > 0 : log.info(check_dict["     failed warning     "])

# log_writer.write() --> ERRORs when checking 'image_name'
if failed_cnt == -1 : print("| {} \n".format(check_dict["  len(image_name_list)  "].strip()))
if failed_cnt > 0: print("| {} \n".format(check_dict["     failed warning     "].strip()))

| 2023-02-28 08:42:13,875 | {Test}_RGB_preprocess | INFO |       #### ERROR : image_name, At least 1 test failed: spelling of 'RGB' 


failed_cnt = 1
check_dict \
{
  "    format_and_type     ": "new_rgb",
  "  len(image_name_list)  ": "PASS",
  "  spelling of 'Series'  ": "PASS",
  "      Series[num]       ": "PASS",
  "   spelling of 'fish'   ": "PASS",
  "       fish_[ID]        ": "PASS",
  "         A or P         ": "PASS",
  "   spelling of 'RGB'    ": "FAILED --> 'RG', misspelling of 'RGB' ",
  "     failed warning     ": "      #### ERROR : image_name, At least 1 test failed: spelling of 'RGB' "
}
| #### ERROR : image_name, At least 1 test failed: spelling of 'RGB' 



# Start Process

In [7]:
def single_RGB_preprocess(lif_path:str, total_lif_file:int, out_dir_root:str, log:logging.Logger, log_writer:TextIO, 
                          Kuwahara_sampleing:int, bf_zproj_type:str, bf_treshold_value:int, relative_mask:str):
    
    
    Reader.setId(lif_path)
    log.info(f'LIF_FILE : {lif_path}')
    
    seriesCount = Reader.getSeriesCount()
    # log.info(f'seriesCount : {seriesCount}')
    
    
    for idx in range(seriesCount):
        
        SeriesNum = idx+1
        
        ij.IJ.run("Bio-Formats Importer", f"open='{lif_path}' color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT series_{SeriesNum}")
        img = ij.WindowManager.getCurrentImage() # get image, <java class 'ij.ImagePlus'>
        # dump_info(img)
        # img.show()
        img.hide()
        
        
        # Get names and image dimensions
        
        # Consociate "palmskin_RGB_RAW" file_name
        file_name = lif_path.split(os.sep)[-1].split(".")[0]
        file_name_list = re.split(" |_|-", file_name)
        assert len(file_name_list) == 4, f"file_name format error, current : '{file_name}', expect like : '20221125_AI005_palmskin_10dpf.lif'"
        file_name = "_".join(file_name_list)
        
        # Get Prop("Image name")
        image_name = img.getProp("Image name")
        image_name = str(image_name)
        
        # Check format of 'image_name'
        detect_old_new_list = re.findall("[RGB]", image_name)
        # print(detect_old_new_list)
        image_name_list = re.split(" |_|-", image_name)
        if len(detect_old_new_list) > 0: failed_cnt, check_dict = check_image_name(image_name_list, "new_rgb")
        else: failed_cnt, check_dict = check_image_name(image_name_list, "old_rgb")
        
        # Consociate image_name
        if (check_dict["    format_and_type     "] == "old_rgb") and (failed_cnt == 0): # name check passed, generate new name
            image_name_list.pop(3) # palmskin (old name only)
            image_name_list.pop(3) # [num]dpf (old name only)
            image_name_list.append("RGB")
        image_name = "_".join(image_name_list)
        
        # Combine 2 names above
        seN = f"{file_name} - {image_name}"
        img_dimensions = img.getDimensions()
        log.info(f"series {SeriesNum:{len(str(seriesCount))}}/{seriesCount} : '{seN}' , Dimensions : {img_dimensions} ( width, height, channels, slices, frames )")
        
        # Print ERRORs when checking 'image_name'
        if failed_cnt == -1 : log.info(check_dict["  len(image_name_list)  "])
        if failed_cnt > 0 : log.info(check_dict["     failed warning     "])
        
        # Write Log
        log_writer.write(f"|-- processing ...  series {SeriesNum:{len(str(seriesCount))}}/{seriesCount} in {SeriesNum}/{total_lif_file} \n")
        log_writer.write(f"|         {seN} \n")
        log_writer.write(f"|         Dimensions : {img_dimensions} ( width, height, channels, slices, frames ) \n")
        if failed_cnt == -1 : log_writer.write("| {} \n".format(check_dict["  len(image_name_list)  "].strip()))
        if failed_cnt > 0: log_writer.write("| {} \n".format(check_dict["     failed warning     "].strip()))
        
        
        # Start image preprocessing
        
        
        # preprocessing var
        # Kuwahara_sampleing = 15
        # bf_zproj_type = "median" # 'method' is "avg", "min", "max", "sum", "sd" or "median".
        # bf_treshold_value = 10
        # relative_mask = "outer" # "inner" or "outer"
        tif_save_cnt = 1
        tif_save_SNdigits = "02"
        
        
        # Create sub folder
        if "delete" in seN: subfolder = os.path.join(out_dir_root, "!~delete", seN)
        else: subfolder = os.path.join(out_dir_root, seN)
        dir_metaimg = os.path.join(subfolder, "MetaImage")
        create_new_dir(subfolder, display_in_CLI=False)
        create_new_dir(dir_metaimg, display_in_CLI=False)
        
        
        ij.IJ.run(img, "Set Scale...", "distance=2.2 known=1 unit=micron")
        # img.show()
        list = channelSplitter.split(img)
        
        
        ch_B = list[0]
        B_processed = median_R1_and_mean3D_R2(ch_B, zProjector, ij)
        tif_save_cnt = save_tif_with_SN(B_processed, "B_processed", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # Kuwahara Filter
        B_processed_Kuwahara = B_processed.duplicate()
        ij.IJ.run(B_processed_Kuwahara, "Kuwahara Filter", f"sampling={Kuwahara_sampleing}")
        tif_save_cnt = save_tif_with_SN(B_processed_Kuwahara, f"B_processed_Kuwahara{Kuwahara_sampleing}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        ch_G = list[1]
        G_processed = median_R1_and_mean3D_R2(ch_G, zProjector, ij)
        tif_save_cnt = save_tif_with_SN(G_processed, "G_processed", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # Kuwahara Filter
        G_processed_Kuwahara = G_processed.duplicate()
        ij.IJ.run(G_processed_Kuwahara, "Kuwahara Filter", f"sampling={Kuwahara_sampleing}")
        tif_save_cnt = save_tif_with_SN(G_processed_Kuwahara, f"G_processed_Kuwahara{Kuwahara_sampleing}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        ch_R = list[2]
        R_processed = median_R1_and_mean3D_R2(ch_R, zProjector, ij)
        tif_save_cnt = save_tif_with_SN(R_processed, "R_processed", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        # Kuwahara Filter
        R_processed_Kuwahara = R_processed.duplicate()
        ij.IJ.run(R_processed_Kuwahara, "Kuwahara Filter", f"sampling={Kuwahara_sampleing}")
        tif_save_cnt = save_tif_with_SN(R_processed_Kuwahara, f"R_processed_Kuwahara{Kuwahara_sampleing}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # Merge, Enhance Contrast, without Kuwahara Filter
        # group_show(R_processed, G_processed, B_processed) # needs to show image before merge
        # ij.IJ.run("Merge Channels...", "c1=[05_R_processed.tif] c2=[03_G_processed.tif] c3=[01_B_processed.tif] create keep")
        # stack_RGB = ij.WindowManager.getCurrentImage()
        stack_RGB = rgbStackMerge.mergeChannels([R_processed, G_processed, B_processed], True)
        # group_hide(R_processed, G_processed, B_processed)
        # ij.IJ.run(stack_RGB, "RGB Color", "") # will create new window
        # composite_RGB_HE = ij.WindowManager.getCurrentImage()
        rgbStackConverter.convertToRGB(stack_RGB)
        tif_save_cnt = save_tif_with_SN(stack_RGB, "composite_RGB", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        composite_RGB_HE = stack_RGB.duplicate()
        ij.IJ.run(composite_RGB_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(composite_RGB_HE, "composite_RGB_HE", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        # group_hide(stack_RGB, composite_RGB_HE)
        
        
        # Merge, Enhance Contrast, with Kuwahara Filter
        # group_show(R_processed_Kuwahara, G_processed_Kuwahara, B_processed_Kuwahara) # needs to show image before merge
        # ij.IJ.run("Merge Channels...", f"c1=[06_R_processed_Kuwahara{Kuwahara_sampleing}.tif] c2=[04_G_processed_Kuwahara{Kuwahara_sampleing}.tif] c3=[02_B_processed_Kuwahara{Kuwahara_sampleing}.tif] create keep")
        # stack_RGB_Kuwahara = ij.WindowManager.getCurrentImage()
        stack_RGB_Kuwahara = rgbStackMerge.mergeChannels([R_processed_Kuwahara, G_processed_Kuwahara, B_processed_Kuwahara], True)
        # group_hide(R_processed_Kuwahara, G_processed_Kuwahara, B_processed_Kuwahara)
        # ij.IJ.run(stack_RGB_Kuwahara, "RGB Color", "") # will create new window
        # composite_RGB_Kuwahara_HE = ij.WindowManager.getCurrentImage()
        rgbStackConverter.convertToRGB(stack_RGB_Kuwahara)
        tif_save_cnt = save_tif_with_SN(stack_RGB_Kuwahara, "composite_RGB_Kuwahara", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        composite_RGB_Kuwahara_HE = stack_RGB_Kuwahara.duplicate()
        ij.IJ.run(composite_RGB_Kuwahara_HE, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(composite_RGB_Kuwahara_HE, "composite_RGB_Kuwahara_HE", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        # group_hide(stack_RGB_Kuwahara, composite_RGB_Kuwahara_HE)
        
        
        # Average with and without Kuwahara Filter
        composite_HE_mix = imageCalculator.run(composite_RGB_HE, composite_RGB_Kuwahara_HE, "Average create")
        tif_save_cnt = save_tif_with_SN(composite_HE_mix, "composite_HE_mix", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # Generate Mask using Bright Field
        ch_BF = list[3]
        BF_Zproj = zProjector.run(ch_BF, bf_zproj_type) # 'method' is "avg", "min", "max", "sum", "sd" or "median".
        tif_save_cnt = save_tif_with_SN(BF_Zproj, f"BF_Zproj_{bf_zproj_type}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        ij.IJ.run(BF_Zproj, "Enhance Contrast...", "saturated=0.35 equalize")
        tif_save_cnt = save_tif_with_SN(BF_Zproj, f"BF_Zproj_{bf_zproj_type}_HE", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        ij.IJ.setRawThreshold(BF_Zproj, 0, bf_treshold_value)
        ij.IJ.run(BF_Zproj, "Convert to Mask", "")
        tif_save_cnt = save_tif_with_SN(BF_Zproj, f"Threshold_0_{bf_treshold_value}", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        threshold_img = imread(f"{dir_metaimg}/{tif_save_cnt-1:{tif_save_SNdigits}}_Threshold_0_{bf_treshold_value}.tif") # use scikit-image
        outer_rect = create_rect(threshold_img, "outer") # use scikit-image
        inner_rect = create_rect(threshold_img, "inner") # use scikit-image
        outer_rect = ij.py.to_imageplus(outer_rect)
        inner_rect = ij.py.to_imageplus(inner_rect)
        tif_save_cnt = save_tif_with_SN(outer_rect, f"outer_rect", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        tif_save_cnt = save_tif_with_SN(inner_rect, f"inner_rect", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        if relative_mask == "outer": mask_rect = outer_rect
        else: mask_rect = inner_rect
        mask_rect = ij.py.to_imageplus(mask_rect)
        ij.IJ.run("Set Measurements...", "area mean min feret's display redirect=None decimal=2")
        ij.IJ.run(mask_rect, "Analyze Particles...", "size=0-Infinity show=Nothing display include add")
        
        
        # Check ROI
        rm.runCommand("Show All with labels")
        rm_size = int(rm.getCount())
        if rm_size == 1:
            rm.save(os.path.normpath(f"{dir_metaimg}/RoiSet.zip"))
        else:
            log.info(f'      ROI in RoiManager: {rm_size}')
            log.info(f'      #### ERROR : Number of ROI not = 1')
            # Write Log
            log_writer.write(f"|         number of ROI = {rm_size} \n")
            log_writer.write("| #### ERROR : Number of ROI not = 1 \n") # ERROR:
        
        
        # # Fit ROI with Rect 
        # mask_rect = mask_img.duplicate()
        # ij.IJ.setForegroundColor(255, 255, 255)
        # ij.IJ.setBackgroundColor(0, 0, 0)
        # mask_rect.show()
        # mask_rect.setRoi(rm.getRoi(0))
        # ij.IJ.run(mask_rect, "Fit Rectangle", "")
        # ij.IJ.run(mask_rect, "Fill", "slice")
        # mask_rect.hide()
        
        
        # Average with Mask_Rect
        composite_RGB_HE_AvgMaskRect = imageCalculator.run(composite_RGB_HE, mask_rect, "Average create")
        composite_RGB_Kuwahara_HE_AvgMaskRect = imageCalculator.run(composite_RGB_Kuwahara_HE, mask_rect, "Average create")
        composite_HE_mix_AvgMaskRect = imageCalculator.run(composite_HE_mix, mask_rect, "Average create")
        tif_save_cnt = save_tif_with_SN(composite_RGB_HE_AvgMaskRect, f"composite_RGB_HE--MIX", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        tif_save_cnt = save_tif_with_SN(composite_RGB_Kuwahara_HE_AvgMaskRect, f"composite_RGB_Kuwahara_HE--MIX", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        tif_save_cnt = save_tif_with_SN(composite_HE_mix_AvgMaskRect, f"composite_HE_mix--MIX", dir_metaimg, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # crop_threshold_roi, save_as_tiff
        cropped_composite_RGB_HE = crop_threshold_rect(composite_RGB_HE, rm.getRoi(0), ij)
        cropped_composite_RGB_Kuwahara_HE = crop_threshold_rect(composite_RGB_Kuwahara_HE, rm.getRoi(0), ij)
        cropped_composite_HE_mix = crop_threshold_rect(composite_HE_mix, rm.getRoi(0), ij)
        tif_save_cnt = save_tif_with_SN(cropped_composite_RGB_HE, f"cropped_composite_RGB_HE", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        tif_save_cnt = save_tif_with_SN(cropped_composite_RGB_Kuwahara_HE, f"cropped_composite_RGB_Kuwahara_HE", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        tif_save_cnt = save_tif_with_SN(cropped_composite_HE_mix, f"cropped_composite_HE_mix", subfolder, tif_save_cnt, tif_save_SNdigits, ij)
        
        
        # Show image
        # composite_HE_mix_AvgMaskRect.show()
        # composite_HE_mix_AvgMaskRect.hide()
        
        rm.runCommand("Deselect")
        rm.runCommand("Delete")
        ij.IJ.run("Clear Results", "")
        ij.IJ.run("Close All", "")
        log_writer.write(f"| \n") # make Log file looks better.

In [8]:
if int(rm.getCount()) > 0:
    rm.runCommand("Deselect")
    rm.runCommand("Delete")
    ij.IJ.run("Clear Results", "")
ij.IJ.run("Close All", "")


preprocess_kwargs = {
    "Kuwahara_sampleing": 15,
    "bf_zproj_type": "min", # 'method' is "avg", "min", "max", "sum", "sd" or "median".
    "bf_treshold_value": 10,
    "relative_mask": "outer", # "inner" or "outer"
}

# File Input
rgb_raw_lif_source = r"C:\Users\confocal_microscope\Desktop\{NAS_DL}_Academia_Sinica_Data\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\palmskin_RGB_RAW"

# Result Output
ap_data = r"C:\Users\confocal_microscope\Desktop\{PyIJ_OutTest}_RGB_preprocess" # r"C:\Users\confocal_microscope\Desktop\WorkingDir\(D2)_Image_AP\{Data}_Data\{20221209_UPDATE_82}_Academia_Sinica_i324"
preprocess_method_desc = "ch4_{}_proj, {}_rect".format(preprocess_kwargs["bf_zproj_type"], preprocess_kwargs["relative_mask"])
preprocess_root = os.path.join(ap_data, f"{{{preprocess_method_desc}}}_RGB_preprocess")
create_new_dir(preprocess_root, display_in_CLI=False)


# Scan Leica LIF file
lif_path_list  = glob(os.path.normpath(f"{rgb_raw_lif_source}/*/*.lif"))
# lif_path_list  = glob(os.path.normpath(f"{bf_raw_lif_source}/*.lif"))
log.info(f'lif_path_list {type(lif_path_list)}: {lif_path_list}, [ found {len(lif_path_list)} files ] \n')


# Create log writer
time_stamp = datetime.now().strftime('%Y%m%d_%H_%M_%S')
log_path = os.path.join(preprocess_root, f"{{Logs}}_{{imagej_RGB_preprocess}}_{time_stamp}.log")
log_writer = open(log_path, mode="w")
log.info(f'log_writer {type(log_writer)}: {log_writer} \n')


# {test} single RGB raw lif
single_lif_path =  r"C:\Users\confocal_microscope\Desktop\{NAS_DL}_Academia_Sinica_Data\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\palmskin_RGB_RAW\Before_20221109\20220610 CE001 palmskin_8dpf.lif"


for i, lif_path in enumerate(lif_path_list): 
    
    log.info(f"Processing ... {i+1}/{len(lif_path_list)}")
    
    # Write Log
    log_writer.write("\n\n\n")
    log_writer.write(f"|-----------------------------------------  Processing ... {i+1}/{len(lif_path_list)}  ----------------------------------------- \n")
    log_writer.write(f"| \n")
    log_writer.write(f"|         LIF_FILE : {lif_path.split(os.sep)[-1]} \n")
    log_writer.write(f"| \n")
    
    # process current LIF_FILE
    single_RGB_preprocess(lif_path, len(lif_path_list), preprocess_root, log, log_writer, **preprocess_kwargs)
    log.info("\n") # make CLI output looks better.


log.info(" -- finished --")
log_writer.write("\n\n\n-----------------------------------------  finished  ----------------------------------------- \n")
log_writer.close()
# rm.close()
# ij.WindowManager.closeAllWindows()

| 2023-02-28 08:42:23,735 | {Test}_RGB_preprocess | INFO | lif_path_list <class 'list'>: ['C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin_RGB_RAW\\Before_20221109\\20220610 CE001 palmskin_8dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin_RGB_RAW\\Before_20221109\\20220613 CE001 palmskin_11dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin_RGB_RAW\\Before_20221109\\20220617 CE002 palmskin_8dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin_RGB_RAW\\Before_20221109\\20220620 CE002 palmskin_11dpf.lif', 'C:\\Users\\confocal_microscope\\Desktop\\{NAS_DL}_Academia_Sinica_Data\\{20230213_DirNameAdjust}_Zebrafish_A_P_strategies\\palmskin

IndexError: arrays used as indices must be of integer (or boolean) type

In [9]:
log_writer.close()